Processing of Depressed- Resting Imperial 

The source dataset is a bit of a nightmare. I used the continuos one. 

The data are in data_clip. trials. However, there are different numbers of trials (up to 400) that are stored in different cells in the trials path. In those datasets the rows are the channels and the column the time. Therefore, each of the trials has to be extracted, transposed and then concatenated with the other trials. The channels are in a separate channel file where the labels are listed in one row under dataclip.label. Then this needs to be brought into the 3d array. 

Problem is:
- the recordings are of different length. Up until now I cropped them to the minimum length to put them into the array (con:loosing data)
- the information of the markers was written for the epoched files, therefore we cannot just determine from the timepoints when something happenened. The epoched data is stored in epochs of consuistent lengths so we can infer the value like that.

In [ ]:
import os
from scipy.io import loadmat
import numpy as np
import pandas as pd

folder = r'C:\Users\s2364956\Desktop\Data\ComplexSystems\Ketamine\Depressed\Resting_imperial'
all_eeg_data = []  # List to store EEG data for all recordings
filenames = []

for fname in os.listdir(folder):
    if not (fname.endswith('.mat') and 'continuous' in fname.lower()):
        continue
    
    path = os.path.join(folder, fname)
    try:
        mat = loadmat(path, struct_as_record=False, squeeze_me=True)
        
        # Extract relevant data
        data = mat['data_clip']
        trials = data.trial  # This is a cell array where each cell is a trial
        times = data.time  # Time corresponding to each trial (n_samples,)

        # Prepare a list to store all trials' data within this recording
        trials_data = []

        # Process each trial (trials is a list of cells, each containing trial data)
        for trial_idx, trial in enumerate(np.atleast_1d(trials)):
            if trial.size == 0:
                continue
            
            # Each trial is a (n_channels, n_samples) array.
            # Transpose it to have time as rows and channels as columns.
            trial_data = trial.T  # Shape will be (n_samples, n_channels) -> (n_timepoints, n_channels)

            # Add the trial data to the list
            trials_data.append(trial_data)

        # Stack all trials within the current recording (along the time axis)
        eeg_data_recording = np.vstack(trials_data)  # Stack vertically (along time dimension)

        # Append this recording's data to the list of all EEG data
        all_eeg_data.append(eeg_data_recording)
        filenames.append(fname)

    except Exception as e:
        print(f"Failed to load {fname}: {e}")


# After processing all files, checkinglengths 
for idx, recording in enumerate(all_eeg_data):
    n_samples = recording.shape[0]  # Number of time points (rows)
    print(f"Recording {filenames[idx]}: {n_samples} samples")





# Find minimum number of samples across all recordings
min_samples = min(recording.shape[0] for recording in all_eeg_data)
print(f"Minimum number of samples across recordings: {min_samples}")

# Trim each recording to the minimum number of samples
all_eeg_data_trimmed = [rec[:min_samples, :] for rec in all_eeg_data]

# Now you can safely convert to 3D array
eeg_3d = np.stack(all_eeg_data_trimmed, axis=0)

print(f"\nTrimmed EEG Data Shape (Recordings × Time/Samples × Channels): {eeg_3d.shape}")


restacking so the channels are in the 2rd dimension of the array

In [ ]:
# Stack first (time as rows, channels as columns)
eeg_3d = np.stack(all_eeg_data_trimmed, axis=0)  # shape (29, min_samples, 63)

# Swap axes to (29, 63, min_samples)
eeg_3d = np.transpose(eeg_3d, (0, 2, 1))

print(f"Final EEG shape: {eeg_3d.shape}")